# Gallery Example: Erl/D/k Queue

This example demonstrates an Erlang/D/k queueing system:
- **Arrivals**: Erlang distribution
- **Service**: Deterministic (constant service times)
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

This multiserver queue combines Erlang arrivals (less variable than Poisson) with deterministic service times (zero variance).

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_erldk(k=2):
    """Create Erl/D/k queueing model"""
    model = Network('Erl/D/k')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    # Erlang arrival with mean=1 and order=5
    source.set_arrival(oclass, Erlang.fit_mean_and_order(1, 5))
    # Deterministic service with mean=2/k (so total capacity matches arrival rate)
    queue.set_service(oclass, Det(2/k))
    queue.set_number_of_servers(k)
    
    # Block 3: topology
    model.link(Network.serial_routing(source, queue, sink))
    
    return model

# Create the model with k=2 servers
model = gallery_erldk(k=2)

## About Erl/D/k Queue

For this queue with k=2 servers:
- **Arrival rate**: λ = 1 (Erlang with mean=1)
- **Service time per server**: 2/k = 1.0 (deterministic)
- **Total service capacity**: k × (1/service_time) = k × (k/2) = k²/2
- **Utilization**: ρ = λ / (k × μ) = 1 / (2 × 1) = 0.5

The combination of:
- Low arrival variability (Erlang SCV = 0.2)
- Zero service variability (deterministic)
- Multiple servers

Results in excellent queueing performance with minimal waiting times.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)